<a href="https://colab.research.google.com/github/Sujal3141/100-days-of-machine-learning/blob/main/House%20price%20prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
!mkdir ~/.kaggle/

In [4]:
!cp kaggle.json ~/.kaggle/kaggle.json

In [5]:
!kaggle competitions download -c house-prices-advanced-regression-techniques

100% 199k/199k [00:00<00:00, 740kB/s]
100% 199k/199k [00:00<00:00, 739kB/s]


In [6]:
!unzip house-prices-advanced-regression-techniques

Archive:  house-prices-advanced-regression-techniques.zip
  inflating: data_description.txt    
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [7]:
import pandas as pd
import matplotlib as plt
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier

In [29]:
! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip


     - 17.8 MB 25.2 MB/s 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for ydata-profiling: filename=ydata_profiling-0.0.dev0-py2.py3-none-any.whl size=357942 sha256=e45632599a89d6f518ad59364de5376ab864025d4b67e5f0dedb2ac6c901cd01
  Stored in directory: /tmp/pip-ephem-wheel-cache-212kqydl/wheels/07/29/61/f533cc7cbd0a97efb2d1b94d3254a3e859a949367ba842577b
Successfully built ydata-profiling
  Attempting uninstall: ydata-profiling
    Found existing installation: ydata-profiling 4.7.0
    Uninstalling ydata-profiling-4.7.0:
      Successfully uninstalled ydata-profiling-4.7.0


In [43]:
import pandas as pd
import pandas_profiling
from pandas_profiling import ProfileReport
from pandas_profiling.utils.cache import cache_file

In [47]:
profile = ProfileReport(train, title="house Dataset", html={'style': {'full_width': True}}, sort=None)

In [ ]:
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')
x_train = train.drop('SalePrice',axis=1)
y_train = train.iloc[:,-1:]

In [9]:
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [10]:
#simple imputer
trf1 = ColumnTransformer([('imputed_LotFrontage',SimpleImputer(),[3]),
                          ('imputed_Alley',SimpleImputer(strategy = 'most_frequent'),[6]),
                          ('imputed_MasVnrType',SimpleImputer(strategy = 'most_frequent'),[25]),
                          ('imputed_MasVnrArea',SimpleImputer(),[26]),
                          ('imputed_BsmtQual',SimpleImputer(strategy='most_frequent'),[30]),
                          ('imputed_BsmtCond',SimpleImputer(strategy='most_frequent'),[31]),
                          ('imputed_BsmtExposure',SimpleImputer(strategy='most_frequent'),[32]),
                          ('imputed_BsmtFinType1',SimpleImputer(strategy='most_frequent'),[33]),
                          ('imputed_BsmtFinType2',SimpleImputer(strategy='most_frequent'),[35]),
                          ('imputed_Electrical',SimpleImputer(strategy='most_frequent'),[42]),
                          ('imputed_FireplaceQu',SimpleImputer(strategy='most_frequent'),[57]),
                          ('imputed_GarageType',SimpleImputer(strategy='most_frequent'),[60]),
                          ('imputed_GarageQual',SimpleImputer(strategy='most_frequent'),[63]),
                          ('imputed_GarageCond',SimpleImputer(strategy='most_frequent'),[64]),
                          ('imputed_PoolQC',SimpleImputer(strategy='most_frequent'),[72]),
                          ('imputed_Fence',SimpleImputer(strategy='most_frequent'),[73]),
                          ('imputed_MiscFeature',SimpleImputer(strategy='most_frequent'),[74]),


                      ],remainder = 'passthrough')

In [11]:
#Making a list of columns which are of datatypes objects
object_dt = train.select_dtypes(include='object')

In [12]:
list_object_dt = list(object_dt)
k= list_object_dt
for i in range(len(list_object_dt)):
  list_object_dt[i]=train.columns.get_loc(list_object_dt[i])


In [13]:
#vARIABLE ENCODING
trf2 = ColumnTransformer([('encoding_MSZoning',OneHotEncoder(),[2]),
('encoding_Street',OneHotEncoder(),[5]),
('encoding_Alley',OneHotEncoder(),[6]),
('encoding_LotShape',OneHotEncoder(),[7]),
('encoding_LandContour',OneHotEncoder(),[8]),
('encoding_Utilities',OneHotEncoder(),[9]),
('encoding_LotConfig',OneHotEncoder(),[10]),
('encoding_LandSlope',OneHotEncoder(),[11]),
('encoding_Neighborhood',OneHotEncoder(),[12]),
('encoding_Condition1',OneHotEncoder(),[13]),
('encoding_Condition2',OneHotEncoder(),[14]),
('encoding_BldgType',OneHotEncoder(),[15]),
('encoding_HouseStyle',OneHotEncoder(),[16]),
('encoding_RoofStyle',OneHotEncoder(),[21]),
('encoding_RoofMatl',OneHotEncoder(),[22]),
('encoding_Exterior1st',OneHotEncoder(),[23]),
('encoding_Exterior2nd',OneHotEncoder(),[24]),
('encoding_MasVnrType',OneHotEncoder(),[25]),
('encoding_ExterQual',OneHotEncoder(),[27]),
('encoding_ExterCond',OneHotEncoder(),[28]),
('encoding_Foundation',OneHotEncoder(),[29]),
('encoding_BsmtQual',OneHotEncoder(),[30]),
('encoding_BsmtCond',OneHotEncoder(),[31]),
('encoding_BsmtExposure',OneHotEncoder(),[32]),
('encoding_BsmtFinType1',OneHotEncoder(),[33]),
('encoding_BsmtFinType2',OneHotEncoder(),[35]),
('encoding_Heating',OneHotEncoder(),[39]),
('encoding_HeatingQC',OneHotEncoder(),[40]),
('encoding_CentralAir',OneHotEncoder(),[41]),
('encoding_Electrical',OneHotEncoder(),[42]),
('encoding_KitchenQual',OneHotEncoder(),[53]),
('encoding_Functional',OneHotEncoder(),[55]),
('encoding_FireplaceQu',OneHotEncoder(),[57]),
('encoding_GarageType',OneHotEncoder(),[58]),
('encoding_GarageFinish',OneHotEncoder(),[60]),
('encoding_GarageQual',OneHotEncoder(),[63]),
('encoding_GarageCond',OneHotEncoder(),[64]),
('encoding_PavedDrive',OneHotEncoder(),[65]),
('encoding_PoolQC',OneHotEncoder(),[72]),
('encoding_Fence',OneHotEncoder(),[73]),
('encoding_MiscFeature',OneHotEncoder(),[74]),
('encoding_SaleType',OneHotEncoder(),[78]),
('encoding_SaleCondition',OneHotEncoder(),[79])],remainder='passthrough')










In [14]:
trf3 = DecisionTreeClassifier()

In [15]:
pipe = make_pipeline(trf1,trf2,trf3)


In [41]:
pipe.fit(,y_train)

ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: 'TA'

In [ ]:

for i in range(len(list_object_dt)) :
  print(f"('encoding_{list(object_dt)[i]}',OneHotEncoder(),[{list_object_dt[i]}]),")

('encoding_MSZoning',OneHotEncoder(),[2]),
('encoding_Street',OneHotEncoder(),[5]),
('encoding_Alley',OneHotEncoder(),[6]),
('encoding_LotShape',OneHotEncoder(),[7]),
('encoding_LandContour',OneHotEncoder(),[8]),
('encoding_Utilities',OneHotEncoder(),[9]),
('encoding_LotConfig',OneHotEncoder(),[10]),
('encoding_LandSlope',OneHotEncoder(),[11]),
('encoding_Neighborhood',OneHotEncoder(),[12]),
('encoding_Condition1',OneHotEncoder(),[13]),
('encoding_Condition2',OneHotEncoder(),[14]),
('encoding_BldgType',OneHotEncoder(),[15]),
('encoding_HouseStyle',OneHotEncoder(),[16]),
('encoding_RoofStyle',OneHotEncoder(),[21]),
('encoding_RoofMatl',OneHotEncoder(),[22]),
('encoding_Exterior1st',OneHotEncoder(),[23]),
('encoding_Exterior2nd',OneHotEncoder(),[24]),
('encoding_MasVnrType',OneHotEncoder(),[25]),
('encoding_ExterQual',OneHotEncoder(),[27]),
('encoding_ExterCond',OneHotEncoder(),[28]),
('encoding_Foundation',OneHotEncoder(),[29]),
('encoding_BsmtQual',OneHotEncoder(),[30]),
('encoding_Bsm

In [38]:
temp = x_train.drop(columns=['Street','Alley'])


In [40]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [39]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 78 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   LotShape       1460 non-null   object 
 6   LandContour    1460 non-null   object 
 7   Utilities      1460 non-null   object 
 8   LotConfig      1460 non-null   object 
 9   LandSlope      1460 non-null   object 
 10  Neighborhood   1460 non-null   object 
 11  Condition1     1460 non-null   object 
 12  Condition2     1460 non-null   object 
 13  BldgType       1460 non-null   object 
 14  HouseStyle     1460 non-null   object 
 15  OverallQual    1460 non-null   int64  
 16  OverallCond    1460 non-null   int64  
 17  YearBuilt      1460 non-null   int64  
 18  YearRemo